Import libraries

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from keras.callbacks import EarlyStopping

Dataset path (train,test,validation)

In [ ]:
image_size=(224,224)
batch_size=16

train_data=r"D:\ALL\DL\image_identify\dataset\train"
val_data=r"D:\ALL\DL\image_identify\dataset\val"
test_data=r"D:\ALL\DL\image_identify\dataset\test"

Image preprocessing -Augmentation (Train_data )

In [6]:
train_datagen=ImageDataGenerator(rescale=1./255,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode="nearest",
                            brightness_range=[0.5,1.5] )       

In [7]:
data_gen=ImageDataGenerator(rescale=1./255)

In [8]:
train=train_datagen.flow_from_directory(train_data,
                                   target_size=image_size,
                                   batch_size=batch_size,
                                   class_mode="sparse")

print(train.class_indices) 
print(train.samples) 
                                 

Found 456 images belonging to 6 classes.
{'cheetah': 0, 'fox': 1, 'hyena': 2, 'lion': 3, 'tiger': 4, 'wolf': 5}
456


In [9]:
val=data_gen.flow_from_directory(val_data,
                                 target_size=image_size,
                                 batch_size=batch_size,
                                 class_mode="sparse")
print(val.class_indices)    
print(val.samples)                             


Found 78 images belonging to 6 classes.
{'cheetah': 0, 'fox': 1, 'hyena': 2, 'lion': 3, 'tiger': 4, 'wolf': 5}
78


Read Test data

In [10]:
test=data_gen.flow_from_directory(test_data,
                                 target_size=image_size,
                                 batch_size=batch_size,
                                 class_mode="sparse")
print(test.class_indices)
print(test.samples)                                 

Found 72 images belonging to 6 classes.
{'cheetah': 0, 'fox': 1, 'hyena': 2, 'lion': 3, 'tiger': 4, 'wolf': 5}
72


CNN (Layers)

In [11]:
model=Sequential ([ #Input layer
      Conv2D(32,(3,3),input_shape=(image_size[0],image_size[1],3),activation="relu"),
      MaxPooling2D(2,2),

      #hidden Layer
      Conv2D(64,(3,3),activation="relu"),
      MaxPooling2D(2,2),

      Conv2D(128,(3,3),activation="relu"),
      MaxPooling2D(2,2),
       

      Flatten(),
      Dense(256,activation="relu"),
      Dropout(0.3),

      #Output layer
      Dense(6,activation="softmax")
      ])

Compile the model 

In [12]:
model.compile(optimizer="adam",
               loss="sparse_categorical_crossentropy",
               metrics=["accuracy"])

Earlystop to prevent overfitting

In [13]:
early=EarlyStopping(monitor="val_loss",
                    patience=5,
                    restore_best_weights=True,
                   )

In [21]:
model_train=model.fit(train,
                      validation_data=val,
                      epochs=50    #30-67%
                       )                      

Epoch 1/50
29/29 [==============================] - 158s 5s/step - loss: 0.8936 - accuracy: 0.6645 - val_loss: 1.4182 - val_accuracy: 0.5897
Epoch 2/50
29/29 [==============================] - 145s 5s/step - loss: 0.8280 - accuracy: 0.6820 - val_loss: 1.5021 - val_accuracy: 0.4872
Epoch 3/50
29/29 [==============================] - 146s 5s/step - loss: 0.8829 - accuracy: 0.6754 - val_loss: 1.3740 - val_accuracy: 0.5769
Epoch 4/50
29/29 [==============================] - 144s 5s/step - loss: 0.8738 - accuracy: 0.6776 - val_loss: 1.1160 - val_accuracy: 0.6154
Epoch 5/50
29/29 [==============================] - 146s 5s/step - loss: 0.8775 - accuracy: 0.6623 - val_loss: 1.2835 - val_accuracy: 0.5897
Epoch 6/50
29/29 [==============================] - 143s 5s/step - loss: 0.8271 - accuracy: 0.6886 - val_loss: 1.2565 - val_accuracy: 0.5513
Epoch 7/50
29/29 [==============================] - 145s 5s/step - loss: 0.7923 - accuracy: 0.6952 - val_loss: 1.1663 - val_accuracy: 0.6026
Epoch 8/50
29

Model save in Hierarchical file

In [22]:
model.save("wild.h5")

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

Test_data evaluate 

In [ ]:
test_loss,test_accuracy=model.evaluate(test)

print(f"testloss:{test_loss}")
print(f"test accuracy:{test_accuracy}")

5/5 [==============================] - 8s 1s/step - loss: 1.0544 - accuracy: 0.6528
testloss:1.0544321537017822
test accuracy:0.6527777910232544


In [ ]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from keras.models import load_model
import numpy as np

model=load_model("wild.h5")

image_path=r"D:\ALL\DL\image_identify\dataset\test\cheetah\00000096_224resized.png"

img=load_img(image_path,target_size=(224,224))
img_array=img_to_array(img)/255.0
img_expand=np.expand_dims(img_array,axis=0)


In [ ]:
class_names=["Cheetah","Fox","Hyena","Lion","Tiger","Wolf"]

predict=model.predict(img_expand)
classes=np.argmax(predict)

print(f"predicted:{class_names[classes]}")

1/1 [==============================] - 1s 1s/step
predicted:Cheetah
